In [1]:
import pickle
import networkx as nx
import pandas as pd
import numpy as np
import time

In [2]:
with open(r'graph_centrality_codes/nodes_edges_weighted.pickle', 'rb') as handle:
    B_matrix_weighted, node_coordinates_weighted = pickle.load(handle)
    
with open(r'graph_centrality_codes/distance.pickle', 'rb') as handle:
    distance_array = pickle.load(handle)  
    
#distance_array (1st col - distance (m), 2nd col - time (s), 3rd col - time in traffic (s))

In [3]:
with open(r"script_output.pickle","rb") as handle:
    origin_destination_path_list,no_connection_list,origin_destination_path_df,no_connection_df=pickle.load(handle)

In [15]:
# origin_destination_path_df

# 0: origin_id
# 1: destination_id
# 2: path
# 3: S000_adjusted
# 4: SA01_adjusted
# 5: SA02_adjusted
# 6: SA03_adjusted
# 7: SE01_adjusted
# 8: SE02_adjusted
# 9: SE03_adjusted
# 10: SI01_adjusted
# 11: SI02_adjusted
# 12: SI03_adjusted

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,173,"[1, 0, 2, 4, 6, 5, 3, 34, 85, 91, 97, 110, 134...",0.055556,0.0,0.000000,0.055556,0.000000,0.0,0.055556,0.0,0.0,0.055556
1,1,195,"[1, 0, 2, 4, 6, 5, 3, 34, 85, 159, 249, 243, 2...",0.055556,0.0,0.000000,0.055556,0.000000,0.0,0.055556,0.0,0.0,0.055556
2,1,223,"[1, 0, 2, 4, 6, 5, 3, 34, 85, 159, 249, 243, 2...",0.055556,0.0,0.000000,0.055556,0.000000,0.0,0.055556,0.0,0.0,0.055556
3,1,3194,"[1, 10, 13, 15, 17, 22, 25, 28, 46, 57, 94, 10...",0.166667,0.0,0.166667,0.000000,0.166667,0.0,0.000000,0.0,0.0,0.166667
4,1,183,"[1, 10, 13, 15, 17, 22, 25, 28, 46, 57, 94, 10...",0.041667,0.0,0.041667,0.000000,0.000000,0.0,0.041667,0.0,0.0,0.041667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5162826,50121,22847,"[50121, 17745, 17843, 17934, 18131, 18031, 179...",0.500000,0.0,0.500000,0.000000,0.500000,0.0,0.000000,0.0,0.0,0.500000
5162827,50121,23278,"[50121, 17745, 17843, 17934, 18131, 18031, 179...",0.166667,0.0,0.000000,0.166667,0.166667,0.0,0.000000,0.0,0.0,0.166667
5162828,50121,23295,"[50121, 17745, 17843, 17934, 18131, 18031, 179...",0.166667,0.0,0.000000,0.166667,0.166667,0.0,0.000000,0.0,0.0,0.166667
5162829,50121,48260,"[50121, 17745, 17843, 17934, 18131, 18031, 179...",0.166667,0.0,0.000000,0.166667,0.166667,0.0,0.000000,0.0,0.0,0.166667


In [4]:
print('Number of rows in df: ' + str(len(origin_destination_path_list)))

Number of rows in df: 5162831


In [ ]:
# Make a copy of the origin_destination_path_list and origin_destination_path_df

# origin_destination_path_list_copy = origin_destination_path_list.copy()
# origin_destination_path_df_copy = origin_destination_path_df.copy()

In [5]:
# Subset of origin_destination_path_list and origin_destination_path_df

origin_destination_path_list_copy = origin_destination_path_list[0:1000]
origin_destination_path_df_copy = origin_destination_path_df[0:1000]

In [6]:
# Count how many elements there are total in the paths of the origin_destination_path_list_copy

total_elements_copy = 0

for i in range(len(origin_destination_path_list_copy)):
    total_elements_copy += len(origin_destination_path_list_copy[i][2])


# Count how many elements there are total in the paths of the origin_destination_path_list

total_elements = 0

for i in range(len(origin_destination_path_list)):
    total_elements += len(origin_destination_path_list[i][2])


print('Total elements in the paths of the origin_destination_path_list_copy: ' + str(total_elements_copy))
print('Total elements in the paths of the origin_destination_path_list: ' + str(total_elements))

Total elements in the paths of the origin_destination_path_list_copy: 42003
Total elements in the paths of the origin_destination_path_list: 575769651


In [18]:
ratio = total_elements / total_elements_copy

In [14]:
# Rename columns

origin_destination_path_df_copy.columns = ['origin_id', 'destination_id', 'path', 'S000_adjusted', 'SA01_adjusted', 'SA02_adjusted', 'SA03_adjusted', 'SE01_adjusted', 'SE02_adjusted', 'SE03_adjusted', 'SI01_adjusted', 'SI02_adjusted', 'SI03_adjusted']

In [9]:
pairlist_actual=[]

for j in range(len(origin_destination_path_list_copy)):
    path = origin_destination_path_list_copy[j][2]
    for i in range(len(path) - 1):
        pair=(min([path[i],path[i+1]]),max([path[i],path[i+1]]))
        if pair not in pairlist_actual:
            pairlist_actual.append(pair)

print(len(pairlist_actual))

2602


In [10]:
# Initialize array of 0s with 10 columns and add to B_matrix_weighted
zero_columns = np.zeros((B_matrix_weighted.shape[0], 10))
B_matrix_weighted_array = np.hstack((B_matrix_weighted, zero_columns))
B_matrix_weighted_array = B_matrix_weighted_array.astype(float)

In [11]:
print(B_matrix_weighted_array.shape)

(70983, 16)


In [12]:
# B_matrix_weighted_df = pd.DataFrame(B_matrix_weighted_array)
# B_matrix_weighted_df

In [41]:
# Approach 1: Indexing the original B_matrix_weighted in array form (Linear, very slow)

start=time.time()
pairlist_test_1=[]

for index, row in origin_destination_path_df_copy.iterrows():
    path = row['path']
    for i in range(len(path) - 1):
        
        idx = np.where((B_matrix_weighted_array[:, 0] == path[i]) & (B_matrix_weighted_array[:, 1] == path[i+1]))[0]

        if len(idx) > 0:
            B_matrix_weighted_array[idx[0], 6] = B_matrix_weighted_array[idx[0], 6] + row['S000_adjusted']
            B_matrix_weighted_array[idx[0], 7] = B_matrix_weighted_array[idx[0], 7] + row['SA01_adjusted']
            B_matrix_weighted_array[idx[0], 8] = B_matrix_weighted_array[idx[0], 8] + row['SA02_adjusted']
            B_matrix_weighted_array[idx[0], 9] = B_matrix_weighted_array[idx[0], 9] + row['SA03_adjusted']
            B_matrix_weighted_array[idx[0], 10] = B_matrix_weighted_array[idx[0], 10] + row['SE01_adjusted']
            B_matrix_weighted_array[idx[0], 11] = B_matrix_weighted_array[idx[0], 11] + row['SE02_adjusted']
            B_matrix_weighted_array[idx[0], 12] = B_matrix_weighted_array[idx[0], 12] + row['SE03_adjusted']
            B_matrix_weighted_array[idx[0], 13] = B_matrix_weighted_array[idx[0], 13] + row['SI01_adjusted']
            B_matrix_weighted_array[idx[0], 14] = B_matrix_weighted_array[idx[0], 14] + row['SI02_adjusted']
            B_matrix_weighted_array[idx[0], 15] = B_matrix_weighted_array[idx[0], 15] + row['SI03_adjusted']
        elif len(idx) == 0:
            idx = np.where((B_matrix_weighted_array[:, 1] == path[i]) & (B_matrix_weighted_array[:, 0] == path[i+1]))[0]
            if len(idx) > 0:
                B_matrix_weighted_array[idx[0], 6] = B_matrix_weighted_array[idx[0], 6] + row['S000_adjusted']
                B_matrix_weighted_array[idx[0], 7] = B_matrix_weighted_array[idx[0], 7] + row['SA01_adjusted']
                B_matrix_weighted_array[idx[0], 8] = B_matrix_weighted_array[idx[0], 8] + row['SA02_adjusted']
                B_matrix_weighted_array[idx[0], 9] = B_matrix_weighted_array[idx[0], 9] + row['SA03_adjusted']
                B_matrix_weighted_array[idx[0], 10] = B_matrix_weighted_array[idx[0], 10] + row['SE01_adjusted']
                B_matrix_weighted_array[idx[0], 11] = B_matrix_weighted_array[idx[0], 11] + row['SE02_adjusted']
                B_matrix_weighted_array[idx[0], 12] = B_matrix_weighted_array[idx[0], 12] + row['SE03_adjusted']
                B_matrix_weighted_array[idx[0], 13] = B_matrix_weighted_array[idx[0], 13] + row['SI01_adjusted']
                B_matrix_weighted_array[idx[0], 14] = B_matrix_weighted_array[idx[0], 14] + row['SI02_adjusted']
                B_matrix_weighted_array[idx[0], 15] = B_matrix_weighted_array[idx[0], 15] + row['SI03_adjusted']
        if idx[0]not in pairlist_test_1:
            pairlist_test_1.append(idx[0])

end=time.time()

print(end-start)
print(len(pairlist_test_1))

8.058660984039307
745


In [44]:
print(len(np.unique(pairlist_test_1)))

745


In [45]:
print(sum(B_matrix_weighted_array[:, 6] != 0))

745


In [15]:
# Approach 2: Converting B_matrix_weighted to a dictionary for faster lookups (O(1) lookups, faster)

# Convert B_matrix_weighted_array to a dictionary for faster lookups
B_matrix_weighted_dict = {(row[0], row[1]): row for row in B_matrix_weighted_array}

start = time.time()

for _, row in origin_destination_path_df_copy.iterrows():
    path = row['path']
    for i in range(len(path) - 1):
        pair = (path[i], path[i+1])
        reverse_pair = (path[i+1], path[i])

        if pair in B_matrix_weighted_dict:
            B_matrix_weighted_dict[pair][6:16] += row['S000_adjusted':'SI03_adjusted']
        elif reverse_pair in B_matrix_weighted_dict:
            B_matrix_weighted_dict[reverse_pair][6:16] += row['S000_adjusted':'SI03_adjusted']

# Convert the dictionary back to a numpy array
B_matrix_weighted_array = np.array(list(B_matrix_weighted_dict.values()))

end = time.time()

print(end - start)

23.588637828826904


In [16]:
len(B_matrix_weighted_array[B_matrix_weighted_array[:, 6] != 0])

2602

In [19]:
# Estimated time to run the entire origin_destination_path_df

test_time = (end - start) * ratio

323348.85056469927

In [22]:
print(test_time / 60 / 60 / 24)

3.742453703703704

In [33]:
# # Approach 2: Indexing the original B_matrix_weighted in dataframe form (This would be slower than the first approach but is working)

# This iterates over each row of the origin_destination_path_df dataframe and for each row, it iterates over each pair of adjacent nodes in the path column.
# For each pair of nodes, it checks if the nodes are present in the first and second columns (or reversed) of the B_matrix_weighted_df dataframe.
# If the nodes are present, it adds the S000_adjusted value to the 6th column of the B_matrix_weighted_df dataframe.

# start=time.time()

# for index, row in origin_destination_path_df.iterrows():
#     for i in range(len(row['path']) - 1):
#         if (row['path'][i] in B_matrix_weighted_df[0] and row['path'][i + 1] in B_matrix_weighted_df[1]):
#             B_matrix_weighted_df.loc[(B_matrix_weighted_df[0] == row['path'][i]) & (B_matrix_weighted_df[1] == row['path'][i + 1]), 6] += row['S000_adjusted']
#             # print(B_matrix_weighted_df.loc[(B_matrix_weighted_df[0] == row['path'][i]) & (B_matrix_weighted_df[1] == row['path'][i + 1])])
#         elif (row['path'][i+1] in B_matrix_weighted_df[0] and row['path'][i] in B_matrix_weighted_df[1]):
#             B_matrix_weighted_df.loc[(B_matrix_weighted_df[1] == row['path'][i]) & (B_matrix_weighted_df[0] == row['path'][i + 1]), 6] += row['S000_adjusted']
#             # print(B_matrix_weighted_df.loc[(B_matrix_weighted_df[0] == row['path'][i]) & (B_matrix_weighted_df[1] == row['path'][i + 1])])

# end=time.time()

# print(end-start)

/var/folders/6m/88dwrhnx7m3cybxwl1p0rtq40000gn/T/ipykernel_95611/2527138943.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.05555556]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  B_matrix_weighted_df.loc[(B_matrix_weighted_df[0] == row['path'][i]) & (B_matrix_weighted_df[1] == row['path'][i + 1]), 6] += row['S000_adjusted']


112.89699602127075


In [34]:
# print(len(B_matrix_weighted_df[B_matrix_weighted_df[6] != 0]))

1006


In [43]:
# B_matrix_weighted_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,31017,30686,640120,684,24,55,0.0,0,0,0,0,0,0,0,0,0
1,40651,40669,3976160,340,28,23,0.0,0,0,0,0,0,0,0,0,0
2,15202,15358,1417450,642,36,322,0.0,0,0,0,0,0,0,0,0,0
3,15276,15427,1417420,642,36,261,0.0,0,0,0,0,0,0,0,0,0
4,15358,15473,1417460,294,36,83,0.0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70978,37860,37869,4728880,195,36,6,0.0,0,0,0,0,0,0,0,0,0
70979,37860,37862,4728870,195,36,1,0.0,0,0,0,0,0,0,0,0,0
70980,37856,37860,4728890,166,36,9,0.0,0,0,0,0,0,0,0,0,0
70981,37877,37850,309910,407,36,0,0.0,0,0,0,0,0,0,0,0,0


In [55]:
print('Number of node pairs updated in B_matrix_weighted: ' + str(len(B_matrix_weighted_array[B_matrix_weighted_array[:, 6] != 0])))

Number of node pairs updated in B_matrix_weighted: 745


In [57]:
# Convert B_matrix_weighted_df back to a numpy array

# B_matrix_weighted_updated = B_matrix_weighted_df.to_numpy()

# Export B_matrix_weighted_updated as a pickle file

with open('B_matrix_weighted_updated.pickle', 'wb') as handle:
    pickle.dump(B_matrix_weighted_array, handle, protocol=pickle.HIGHEST_PROTOCOL)